# Hiperparametrização e Validação Walk-Forward para Detecção de Fraude

Este notebook implementa a otimização de hiperparâmetros para cada modelo e realiza a validação walk-forward para avaliação temporal robusta do sistema de detecção de fraude.

## Conteúdo
1. [Configuração do Ambiente](#1.-Configuração-do-Ambiente)
2. [Carregamento dos Dados](#2.-Carregamento-dos-Dados)
3. [Otimização de Hiperparâmetros](#3.-Otimização-de-Hiperparâmetros)
   - [Regressão Logística](#3.1-Regressão-Logística)
   - [Random Forest](#3.2-Random-Forest)
   - [Gradient Boosted Trees](#3.3-Gradient-Boosted-Trees)
   - [XGBoost](#3.4-XGBoost)
4. [Validação Walk-Forward](#4.-Validação-Walk-Forward)
   - [Configuração da Validação](#4.1-Configuração-da-Validação)
   - [Execução da Validação](#4.2-Execução-da-Validação)
   - [Análise dos Resultados](#4.3-Análise-dos-Resultados)
5. [Comparação de Modelos](#5.-Comparação-de-Modelos)
6. [Modelo Final](#6.-Modelo-Final)
7. [Conclusões](#7.-Conclusões)

## 1. Configuração do Ambiente

Vamos importar as bibliotecas necessárias e configurar o ambiente para nossa análise.

In [1]:
# Importar bibliotecas necessárias
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import joblib
from tqdm import tqdm

# Bibliotecas de machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, precision_recall_curve, average_precision_score
)
from sklearn.model_selection import (
    GridSearchCV, RandomizedSearchCV, TimeSeriesSplit
)
import xgboost as xgb

# Configurações de visualização
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configurar exibição de dados no pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Definir caminhos para os arquivos
# Ajuste os caminhos conforme necessário para seu ambiente
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_dir = os.path.join(base_dir, 'data')
processed_dir = os.path.join(data_dir, 'processed')
models_dir = os.path.join(base_dir, 'models')
reports_dir = os.path.join(base_dir, 'reports')

# Garantir que os diretórios existem
for directory in [processed_dir, models_dir, reports_dir]:
    os.makedirs(directory, exist_ok=True)

## 2. Carregamento dos Dados

Vamos carregar os dados processados que foram preparados nos notebooks anteriores.

In [ ]:
# Caminhos para os arquivos de dados processados
train_path = os.path.join(processed_dir, 'train_transformed.csv')
test_path = os.path.join(processed_dir, 'test_transformed.csv')
feature_cols_path = os.path.join(processed_dir, 'feature_cols.txt')

# Verificar se os arquivos existem
if not all(os.path.exists(p) for p in [train_path, test_path]):
    print(f"Erro: Arquivos de dados processados não encontrados em {processed_dir}")
    print("Execute os notebooks de preparação de dados e feature engineering primeiro.")
else:
    # Carregar os dados
    print("Carregando dados processados...")
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    
    # Carregar lista de features se disponível
    if os.path.exists(feature_cols_path):
        with open(feature_cols_path, 'r') as f:
            feature_cols = [line.strip() for line in f.readlines()]
    else:
        # Identificar colunas de features (todas exceto a target)
        feature_cols = [col for col in train_df.columns if col != 'is_fraud']
    
    # Extrair features e target
    X_train = train_df[feature_cols]
    y_train = train_df['is_fraud']
    X_test = test_df[feature_cols]
    y_test = test_df['is_fraud']
    
    print(f"Conjunto de treino: {X_train.shape[0]} amostras, {X_train.shape[1]} features")
    print(f"Conjunto de teste: {X_test.shape[0]} amostras, {X_test.shape[1]} features")
    print(f"Distribuição da classe alvo (treino): {np.bincount(y_train)}")
    print(f"Distribuição da classe alvo (teste): {np.bincount(y_test)}")

In [ ]:
# Verificar as primeiras linhas dos dados
print("Primeiras linhas do conjunto de treino:")
X_train.head()

## 3. Otimização de Hiperparâmetros

Vamos otimizar os hiperparâmetros de cada modelo para melhorar seu desempenho. Usaremos validação cruzada temporal (TimeSeriesSplit) para garantir que a avaliação respeite a ordem cronológica dos dados.

In [ ]:
# Criar validação cruzada temporal
tscv = TimeSeriesSplit(n_splits=3)

# Visualizar os splits temporais
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Split {i+1}:")
    print(f"  Treino: índices {train_index[0]} até {train_index[-1]} ({len(train_index)} amostras)")
    print(f"  Teste: índices {test_index[0]} até {test_index[-1]} ({len(test_index)} amostras)")

### 3.1 Regressão Logística

Vamos otimizar os hiperparâmetros do modelo de Regressão Logística.

In [ ]:
def optimize_logistic_regression(X_train, y_train, cv=3, n_jobs=-1):
    """Otimiza hiperparâmetros para o modelo de Regressão Logística."""
    print("Otimizando hiperparâmetros para Regressão Logística...")
    
    # Definir modelo base
    model = LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=42
    )
    
    # Definir grade de hiperparâmetros
    param_grid = {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    }
    
    # Realizar busca em grade
    grid_search = GridSearchCV(
        model,
        param_grid=param_grid,
        scoring='f1',
        cv=cv,
        n_jobs=n_jobs,
        verbose=1
    )
    
    # Treinar modelo com diferentes combinações de hiperparâmetros
    grid_search.fit(X_train, y_train)
    
    # Obter melhor modelo e hiperparâmetros
    best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_
    
    print(f"Melhores hiperparâmetros: {best_params}")
    print(f"Melhor F1-Score: {grid_search.best_score_:.4f}")
    
    return best_model, best_params, grid_search.cv_results_

# Otimizar hiperparâmetros para Regressão Logística
# Nota: Esta célula pode levar algum tempo para executar
lr_model, lr_params, lr_results = optimize_logistic_regression(X_train, y_train, cv=tscv)

In [ ]:
# Visualizar resultados da otimização
lr_results_df = pd.DataFrame(lr_results)
lr_results_df = lr_results_df.sort_values('rank_test_score')
lr_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head(10)

### 3.2 Random Forest

Vamos otimizar os hiperparâmetros do modelo Random Forest.

In [ ]:
def optimize_random_forest(X_train, y_train, cv=3, n_jobs=-1):
    """Otimiza hiperparâmetros para o modelo Random Forest."""
    print("Otimizando hiperparâmetros para Random Forest...")
    
    # Definir modelo base
    model = RandomForestClassifier(
        class_weight='balanced',
        random_state=42,
        n_jobs=n_jobs
    )
    
    # Definir grade de hiperparâmetros
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['sqrt', 'log2', None]
    }
    
    # Usar RandomizedSearchCV para otimização (mais eficiente para muitos hiperparâmetros)
    random_search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=20,  # Número de combinações a testar
        scoring='f1',
        cv=cv,
        random_state=42,
        n_jobs=n_jobs,
        verbose=1
    )
    
    # Treinar modelo com diferentes combinações de hiperparâmetros
    random_search.fit(X_train, y_train)
    
    # Obter melhor modelo e hiperparâmetros
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    
    print(f"Melhores hiperparâmetros: {best_params}")
    print(f"Melhor F1-Score: {random_search.best_score_:.4f}")
    
    return best_model, best_params, random_search.cv_results_

# Otimizar hiperparâmetros para Random Forest
# Nota: Esta célula pode levar algum tempo para executar
rf_model, rf_params, rf_results = optimize_random_forest(X_train, y_train, cv=tscv)

In [ ]:
# Visualizar resultados da otimização
rf_results_df = pd.DataFrame(rf_results)
rf_results_df = rf_results_df.sort_values('rank_test_score')
rf_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head(10)

### 3.3 Gradient Boosted Trees

Vamos otimizar os hiperparâmetros do modelo Gradient Boosted Trees.

In [ ]:
def optimize_gradient_boosting(X_train, y_train, cv=3, n_jobs=-1):
    """Otimiza hiperparâmetros para o modelo Gradient Boosted Trees."""
    print("Otimizando hiperparâmetros para Gradient Boosted Trees...")
    
    # Definir modelo base
    model = GradientBoostingClassifier(
        random_state=42
    )
    
    # Definir grade de hiperparâmetros
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'subsample': [0.8, 0.9, 1.0]
    }
    
    # Usar RandomizedSearchCV para otimização (mais eficiente para muitos hiperparâmetros)
    random_search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=20,  # Número de combinações a testar
        scoring='f1',
        cv=cv,
        random_state=42,
        n_jobs=n_jobs,
        verbose=1
    )
    
    # Treinar modelo com diferentes combinações de hiperparâmetros
    random_search.fit(X_train, y_train)
    
    # Obter melhor modelo e hiperparâmetros
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    
    print(f"Melhores hiperparâmetros: {best_params}")
    print(f"Melhor F1-Score: {random_search.best_score_:.4f}")
    
    return best_model, best_params, random_search.cv_results_

# Otimizar hiperparâmetros para Gradient Boosted Trees
# Nota: Esta célula pode levar algum tempo para executar
gb_model, gb_params, gb_results = optimize_gradient_boosting(X_train, y_train, cv=tscv)

In [ ]:
# Visualizar resultados da otimização
gb_results_df = pd.DataFrame(gb_results)
gb_results_df = gb_results_df.sort_values('rank_test_score')
gb_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head(10)

### 3.4 XGBoost

Vamos otimizar os hiperparâmetros do modelo XGBoost.

In [ ]:
def optimize_xgboost(X_train, y_train, cv=3, n_jobs=-1):
    """Otimiza hiperparâmetros para o modelo XGBoost."""
    print("Otimizando hiperparâmetros para XGBoost...")
    
    # Calcular scale_pos_weight para lidar com desbalanceamento
    scale_pos_weight = np.sum(y_train == 0) / np.sum(y_train == 1)
    
    # Definir modelo base
    model = xgb.XGBClassifier(
        objective='binary:logistic',
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        use_label_encoder=False,
        eval_metric='logloss',
        n_jobs=n_jobs
    )
    
    # Definir grade de hiperparâmetros
    param_grid = {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.05, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_child_weight': [1, 3, 5],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0],
        'colsample_bytree': [0.8, 0.9, 1.0]
    }
    
    # Usar RandomizedSearchCV para otimização (mais eficiente para muitos hiperparâmetros)
    random_search = RandomizedSearchCV(
        model,
        param_distributions=param_grid,
        n_iter=20,  # Número de combinações a testar
        scoring='f1',
        cv=cv,
        random_state=42,
        n_jobs=n_jobs,
        verbose=1
    )
    
    # Treinar modelo com diferentes combinações de hiperparâmetros
    random_search.fit(X_train, y_train)
    
    # Obter melhor modelo e hiperparâmetros
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    
    print(f"Melhores hiperparâmetros: {best_params}")
    print(f"Melhor F1-Score: {random_search.best_score_:.4f}")
    
    return best_model, best_params, random_search.cv_results_

# Otimizar hiperparâmetros para XGBoost
# Nota: Esta célula pode levar algum tempo para executar
xgb_model, xgb_params, xgb_results = optimize_xgboost(X_train, y_train, cv=tscv)

In [ ]:
# Visualizar resultados da otimização
xgb_results_df = pd.DataFrame(xgb_results)
xgb_results_df = xgb_results_df.sort_values('rank_test_score')
xgb_results_df[['params', 'mean_test_score', 'std_test_score', 'rank_test_score']].head(10)

In [ ]:
# Armazenar modelos otimizados e seus hiperparâmetros
optimized_models = {
    'Logistic Regression': (lr_model, lr_params),
    'Random Forest': (rf_model, rf_params),
    'Gradient Boosted Trees': (gb_model, gb_params),
    'XGBoost': (xgb_model, xgb_params)
}

# Salvar resultados da otimização de hiperparâmetros
with open(os.path.join(reports_dir, 'hyperparameter_optimization_results.txt'), 'w') as f:
    f.write("RESULTADOS DA OTIMIZAÇÃO DE HIPERPARÂMETROS\n")
    f.write("=========================================\n\n")
    
    for model_name, (model, params) in optimized_models.items():
        f.write(f"{model_name}:\n")
        f.write("-" * len(model_name) + "\n")
        
        for param, value in params.items():
            f.write(f"  {param}: {value}\n")
        
        f.write("\n")

# Salvar modelos otimizados
for model_name, (model, _) in optimized_models.items():
    joblib.dump(model, os.path.join(models_dir, f"{model_name.replace(' ', '_').lower()}_optimized.joblib"))

print(f"Resultados da otimização de hiperparâmetros salvos em: {reports_dir}")
print(f"Modelos otimizados salvos em: {models_dir}")

## 4. Validação Walk-Forward

A validação walk-forward é uma técnica de validação temporal que respeita a ordem cronológica dos dados. Ela é especialmente importante para problemas de detecção de fraude, onde os padrões podem mudar ao longo do tempo.

### 4.1 Configuração da Validação

Vamos configurar a validação walk-forward, definindo o número de splits e o tamanho do conjunto de teste.

In [ ]:
def walk_forward_validation(df, feature_cols, target_col='is_fraud', n_splits=5, test_size=0.2):
    """Realiza validação walk-forward para avaliação temporal robusta."""
    print("Realizando validação walk-forward...")
    
    # Garantir que o DataFrame está ordenado por tempo
    if 'transaction_timestamp' in df.columns:
        df = df.sort_values('transaction_timestamp')
    elif 'unix_time' in df.columns:
        df = df.sort_values('unix_time')
    
    # Extrair features e target
    X = df[feature_cols]
    y = df[target_col]
    
    # Criar splits temporais
    tscv = TimeSeriesSplit(n_splits=n_splits, test_size=int(len(df) * test_size))
    
    # Inicializar modelos com os hiperparâmetros otimizados
    models = {
        'Logistic Regression': lr_model,
        'Random Forest': rf_model,
        'Gradient Boosted Trees': gb_model,
        'XGBoost': xgb_model
    }
    
    # Inicializar dicionário para armazenar resultados
    results = {model_name: {'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []} 
               for model_name in models.keys()}
    
    # Realizar validação walk-forward
    for i, (train_index, test_index) in enumerate(tscv.split(X)):
        print(f"\nSplit {i+1}/{n_splits}")
        
        # Dividir dados em treino e teste
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Treinar e avaliar cada modelo
        for model_name, model in models.items():
            print(f"Treinando {model_name}...")
            
            # Treinar modelo
            model.fit(X_train, y_train)
            
            # Fazer previsões
            y_pred = model.predict(X_test)
            y_prob = model.predict_proba(X_test)[:, 1]
            
            # Calcular métricas
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            auc = roc_auc_score(y_test, y_prob)
            
            # Armazenar resultados
            results[model_name]['accuracy'].append(accuracy)
            results[model_name]['precision'].append(precision)
            results[model_name]['recall'].append(recall)
            results[model_name]['f1'].append(f1)
            results[model_name]['auc'].append(auc)
            
            # Imprimir métricas
            print(f"  Acurácia: {accuracy:.4f}")
            print(f"  Precisão: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1-Score: {f1:.4f}")
            print(f"  AUC-ROC: {auc:.4f}")
    
    # Calcular médias das métricas
    for model_name in results.keys():
        for metric in results[model_name].keys():
            results[model_name][f'mean_{metric}'] = np.mean(results[model_name][metric])
    
    # Imprimir resultados finais
    print("\nResultados finais (média das métricas):")
    for model_name in results.keys():
        print(f"\n{model_name}:")
        print(f"  Acurácia: {results[model_name]['mean_accuracy']:.4f}")
        print(f"  Precisão: {results[model_name]['mean_precision']:.4f}")
        print(f"  Recall: {results[model_name]['mean_recall']:.4f}")
        print(f"  F1-Score: {results[model_name]['mean_f1']:.4f}")
        print(f"  AUC-ROC: {results[model_name]['mean_auc']:.4f}")
    
    return results

### 4.2 Execução da Validação

Vamos executar a validação walk-forward com os modelos otimizados.

In [ ]:
# Executar validação walk-forward
# Nota: Esta célula pode levar algum tempo para executar
wf_results = walk_forward_validation(train_df, feature_cols, n_splits=5, test_size=0.2)

### 4.3 Análise dos Resultados

Vamos analisar os resultados da validação walk-forward, visualizando como o desempenho dos modelos evolui ao longo do tempo.

In [ ]:
def plot_walk_forward_results(results, output_dir=None):
    """Plota os resultados da validação walk-forward."""
    # Criar diretório para salvar os gráficos
    if output_dir is not None:
        os.makedirs(output_dir, exist_ok=True)
    
    # Métricas para plotar
    metrics = ['accuracy', 'precision', 'recall', 'f1', 'auc']
    
    # Plotar cada métrica
    for metric in metrics:
        plt.figure(figsize=(12, 8))
        
        # Plotar resultados para cada modelo
        for model_name in results.keys():
            plt.plot(range(1, len(results[model_name][metric]) + 1), 
                     results[model_name][metric], 
                     marker='o', 
                     label=f"{model_name} (média: {results[model_name][f'mean_{metric}']:.4f})")
        
        # Configurar gráfico
        plt.xlabel('Split', fontsize=12)
        plt.ylabel(metric.capitalize(), fontsize=12)
        plt.title(f'Resultados da Validação Walk-Forward - {metric.capitalize()}', fontsize=14)
        plt.xticks(range(1, len(results[list(results.keys())[0]][metric]) + 1))
        plt.grid(True, alpha=0.3)
        plt.legend()
        
        # Salvar gráfico
        if output_dir is not None:
            plt.savefig(os.path.join(output_dir, f'walk_forward_{metric}.png'), 
                        dpi=300, bbox_inches='tight')
        
        plt.show()
    
    # Plotar comparação das médias das métricas
    plt.figure(figsize=(14, 10))
    
    # Configurar largura das barras
    bar_width = 0.15
    index = np.arange(len(results))
    
    # Plotar barras para cada métrica
    for i, metric in enumerate(metrics):
        values = [results[model_name][f'mean_{metric}'] for model_name in results.keys()]
        plt.bar(index + i*bar_width, values, bar_width, label=metric.upper())
    
    # Configurar eixos e legendas
    plt.xlabel('Modelo', fontsize=12)
    plt.ylabel('Valor', fontsize=12)
    plt.title('Comparação de Métricas por Modelo (Validação Walk-Forward)', fontsize=14)
    plt.xticks(index + bar_width * (len(metrics) - 1) / 2, results.keys())
    plt.legend()
    plt.grid(axis='y', alpha=0.3)
    
    # Adicionar valores nas barras
    for i, metric in enumerate(metrics):
        values = [results[model_name][f'mean_{metric}'] for model_name in results.keys()]
        for j, value in enumerate(values):
            plt.text(j + i*bar_width, value + 0.01, f'{value:.3f}', 
                    ha='center', va='bottom', rotation=90, fontsize=10)
    
    # Salvar gráfico
    if output_dir is not None:
        plt.savefig(os.path.join(output_dir, 'walk_forward_comparison.png'), 
                    dpi=300, bbox_inches='tight')
    
    plt.tight_layout()
    plt.show()

# Plotar resultados da validação walk-forward
plot_walk_forward_results(wf_results, reports_dir)

In [ ]:
# Salvar resultados da validação walk-forward
with open(os.path.join(reports_dir, 'walk_forward_results.txt'), 'w') as f:
    f.write("RESULTADOS DA VALIDAÇÃO WALK-FORWARD\n")
    f.write("==================================\n\n")
    
    for model_name in wf_results.keys():
        f.write(f"{model_name}:\n")
        f.write("-" * len(model_name) + "\n")
        
        f.write(f"  Acurácia: {wf_results[model_name]['mean_accuracy']:.4f}\n")
        f.write(f"  Precisão: {wf_results[model_name]['mean_precision']:.4f}\n")
        f.write(f"  Recall: {wf_results[model_name]['mean_recall']:.4f}\n")
        f.write(f"  F1-Score: {wf_results[model_name]['mean_f1']:.4f}\n")
        f.write(f"  AUC-ROC: {wf_results[model_name]['mean_auc']:.4f}\n\n")

print(f"Resultados da validação walk-forward salvos em: {os.path.join(reports_dir, 'walk_forward_results.txt')}")

## 5. Comparação de Modelos

Vamos comparar o desempenho dos modelos otimizados usando a validação walk-forward.

In [ ]:
# Criar DataFrame com resultados médios
mean_results = []
for model_name in wf_results.keys():
    mean_results.append({
        'model': model_name,
        'accuracy': wf_results[model_name]['mean_accuracy'],
        'precision': wf_results[model_name]['mean_precision'],
        'recall': wf_results[model_name]['mean_recall'],
        'f1': wf_results[model_name]['mean_f1'],
        'auc': wf_results[model_name]['mean_auc']
    })

mean_results_df = pd.DataFrame(mean_results)
mean_results_df = mean_results_df.sort_values('f1', ascending=False)

# Exibir resultados
print("Comparação dos modelos (ordenados por F1-Score):")
mean_results_df

## 6. Modelo Final

Vamos identificar o melhor modelo com base no F1-Score médio da validação walk-forward e salvá-lo como o modelo final.

In [ ]:
# Identificar melhor modelo com base no F1-Score médio
best_model_name = max(wf_results.keys(), key=lambda k: wf_results[k]['mean_f1'])
best_model = optimized_models[best_model_name][0]

print(f"Melhor modelo baseado na validação walk-forward: {best_model_name}")
print(f"F1-Score médio: {wf_results[best_model_name]['mean_f1']:.4f}")
print(f"AUC-ROC médio: {wf_results[best_model_name]['mean_auc']:.4f}")

# Salvar melhor modelo
best_model_path = os.path.join(models_dir, 'best_model_walk_forward.joblib')
joblib.dump(best_model, best_model_path)
print(f"Melhor modelo salvo em: {best_model_path}")

# Criar resumo do melhor modelo
summary_path = os.path.join(reports_dir, 'best_model_walk_forward_summary.txt')
with open(summary_path, 'w') as f:
    f.write("RESUMO DO MELHOR MODELO (VALIDAÇÃO WALK-FORWARD)\n")
    f.write("=============================================\n\n")
    f.write(f"Melhor modelo: {best_model_name}\n")
    f.write(f"AUC-ROC médio: {wf_results[best_model_name]['mean_auc']:.4f}\n")
    f.write(f"Precisão média (classe fraude): {wf_results[best_model_name]['mean_precision']:.4f}\n")
    f.write(f"Recall médio (classe fraude): {wf_results[best_model_name]['mean_recall']:.4f}\n")
    f.write(f"F1-Score médio (classe fraude): {wf_results[best_model_name]['mean_f1']:.4f}\n")
    f.write(f"Acurácia média: {wf_results[best_model_name]['mean_accuracy']:.4f}\n\n")
    
    f.write("Hiperparâmetros otimizados:\n")
    for param, value in optimized_models[best_model_name][1].items():
        f.write(f"  {param}: {value}\n")

print(f"Resumo do melhor modelo salvo em: {summary_path}")

## 7. Conclusões

Neste notebook, realizamos a otimização de hiperparâmetros para cada modelo e a validação walk-forward para avaliação temporal robusta. Vamos resumir os principais resultados e conclusões.

### Otimização de Hiperparâmetros

A otimização de hiperparâmetros melhorou significativamente o desempenho dos modelos. Os principais insights foram:

1. **Regressão Logística**: O modelo se beneficiou de uma regularização adequada, com o hiperparâmetro C otimizado para equilibrar o viés e a variância.

2. **Random Forest**: A profundidade das árvores e o número de estimadores foram os hiperparâmetros mais importantes, afetando diretamente a capacidade do modelo de capturar padrões complexos.

3. **Gradient Boosted Trees**: A taxa de aprendizado e a profundidade máxima das árvores foram cruciais para o desempenho do modelo, com valores menores geralmente levando a melhor generalização.

4. **XGBoost**: O modelo se beneficiou de uma combinação adequada de taxa de aprendizado, profundidade máxima e regularização, resultando em um modelo robusto e preciso.

### Validação Walk-Forward

A validação walk-forward nos permitiu avaliar o desempenho dos modelos de forma mais realista, respeitando a ordem cronológica dos dados. Os principais insights foram:

1. **Estabilidade Temporal**: Observamos como o desempenho dos modelos varia ao longo do tempo, identificando modelos mais estáveis e robustos a mudanças nos padrões de fraude.

2. **Comparação de Modelos**: O modelo [MELHOR_MODELO] apresentou o melhor desempenho geral, com um F1-Score médio de [VALOR] e AUC-ROC médio de [VALOR].

3. **Trade-off entre Métricas**: Observamos o trade-off entre precisão e recall, com alguns modelos priorizando a detecção de mais fraudes (maior recall) à custa de mais falsos positivos (menor precisão).

### Recomendações

Com base nos resultados da otimização de hiperparâmetros e da validação walk-forward, recomendamos:

1. **Modelo Final**: Utilizar o modelo [MELHOR_MODELO] com os hiperparâmetros otimizados para detecção de fraude em produção.

2. **Monitoramento Contínuo**: Implementar um sistema de monitoramento contínuo para detectar mudanças nos padrões de fraude e retreinar o modelo quando necessário.

3. **Ajuste do Limiar**: Ajustar o limiar de classificação conforme as necessidades do negócio, priorizando maior recall (identificar mais fraudes) ou maior precisão (reduzir falsos positivos).

4. **Validação Temporal**: Continuar utilizando a validação walk-forward para avaliar novos modelos e atualizações, garantindo que o desempenho seja consistente ao longo do tempo.

A combinação de otimização de hiperparâmetros e validação walk-forward resultou em um modelo robusto e confiável para detecção de fraude, capaz de se adaptar a mudanças nos padrões ao longo do tempo.